In [ ]:
!pip install --upgrade mesa
!pip install --upgrade networkx[default]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import mesa
import networkx as nx

In [ ]:
df = pd.read_csv('data/20230413_clb_taps.csv')
df['Datetime'] = pd.to_datetime(df['Datetime'])
entry_df = df.set_index('Datetime')
entry_df = entry_df[entry_df['Direction']=='Entry']['Direction'].resample('10min',  label='left').count().reset_index()

In [ ]:
entry_df

## Agent-Based Model

In [ ]:
class LibAgent(mesa.Agent):
    """An agent representing library user"""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.timesteps = np.random.uniform(1,20) # time spent studying in library
        self.timesteps = np.random.normal(10,0) # time spent studying in library

    def step(self):
        if self.timesteps > 0:
            self.timesteps -= 1
        else:
            self.model.schedule.remove(self)
            # TODO: remove from grid/network

In [ ]:
def compute_agents(model):
    return len(model.schedule.agents)

class LibModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, entry_df):
        self.schedule = mesa.time.RandomActivation(self)
        self._curr_step = 0
        self.entry_dist = entry_df.to_dict()['Direction']
        self.total_steps = len(self.entry_dist) # total steps taken
        self.datacollector = mesa.DataCollector(
            model_reporters={"num_agents": compute_agents},#  agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        self.datacollector.collect(self)
        # Create agents, where number of agents follows Poisson distribution
        num_agents = self.entry_dist[self._curr_step]
        for i in range(np.random.poisson(num_agents)):
            agent = LibAgent(f'{self._curr_step}-{i}', self)
            # print(agent.unique_id, 'created!') # for debugging
            self.schedule.add(agent) # add to schedule
        self.schedule.step()
        self._curr_step += 1

    def run(self):
        for i in range(self.total_steps):
            self.step()

In [ ]:
model = LibModel(entry_df)
model.run()
results = model.datacollector.get_model_vars_dataframe()
results['Datetime'] = entry_df['Datetime']
sns.lineplot(results, x='Datetime', y='num_agents')

In [ ]:
class LibAgent(mesa.Agent):
    #initialize
    def __init__(self, unique_id, model, year, pref_levels, pref_seat_types, environment_qualities):
        super().__init__(unique_id, model)
        self.year = year
        self.pref_levels = pref_levels #lists
        self.pref_seat_types = pref_seat_types #lists
        self.environment_qualities = environment_qualities # dictionary of{'Privacy' : 0, 'Crowd level' : 0, 'Comfort' : 0, 'Scenery' : 0, 'Lighting' : 0, 'Ease of finding seats' : 0}
        self.timesteps = np.random.uniform(1,20) # time spent studying in library
        self.pos = None
    
    def step(self):
        if self.timestep > 0:
            # Check if the agent is already in a section
            if self.pos is None: #not yet in a section
                self.pos = self.find_section()  # Move the agent to the chosen section
                self.timestep -= 1
            else:
                self.timestep -= 1
                if self.timestep == 0:
                    # Ready to leave library
                    self.model.schedule.remove(self)
    
    def find_section(self):
        